In [1]:
# EE123 Spring 2022 Final Project
# Authors: Brandon VanBuskirk, Steven Foryoung, Matthew Haddad

In [2]:
# we can remove unused imports later. 
import numpy as np
import matplotlib.pyplot as plt
import queue as Queue
import threading
import time
import sys
import multiprocessing

from numpy import ones, zeros, pi, sin, cos, exp, sign, r_, mean, power
from numpy.fft import fft, fftshift, ifft, ifftshift
from scipy import signal
from scipy import integrate

import bitarray
from scipy.io.wavfile import read as wavread
import ax25 as ax25
from math import gcd
import sounddevice as sd
import RPi.GPIO as GPIO
from functools import reduce
import base64

%matplotlib inline
# custom files
from compress_functions import *
from TNCaprs import *

### Pre-task 1
We create an audio loopback with the command
```sh
sudo modprobe snd_aloop
```
You can check that it was successful with
```sh
aplay -L | grep -i plughw:card=loopback
aplay -l | grep -i loopback
```

Run the commands below and ensure the loopback exists.

In [3]:
!sudo modprobe snd_aloop
!aplay -L | grep -i plughw:card=loopback
!aplay -l | grep -i loopback

plughw:CARD=Loopback,DEV=0
plughw:CARD=Loopback,DEV=1
card 2: Loopback [Loopback], device 0: Loopback PCM [Loopback PCM]
card 2: Loopback [Loopback], device 1: Loopback PCM [Loopback PCM]


In [4]:
sd.query_devices()

   0 bcm2835 Headphones: - (hw:0,0), ALSA (0 in, 8 out)
   1 Fe-Pi Audio: HiFi sgtl5000-0 (hw:1,0), ALSA (2 in, 2 out)
   2 Loopback: PCM (hw:2,0), ALSA (32 in, 32 out)
   3 Loopback: PCM (hw:2,1), ALSA (1 in, 1 out)
   4 sysdefault, ALSA (0 in, 128 out)
   5 lavrate, ALSA (0 in, 128 out)
   6 samplerate, ALSA (0 in, 128 out)
   7 speexrate, ALSA (0 in, 128 out)
   8 pulse, ALSA (32 in, 32 out)
   9 upmix, ALSA (0 in, 8 out)
  10 vdownmix, ALSA (0 in, 6 out)
  11 dmix, ALSA (0 in, 2 out)
* 12 default, ALSA (32 in, 32 out)

In [5]:
print(sd.check_output_settings())

None


In [6]:
builtin_idx = 0
usb_idx = 1
loop_idx = 3  # Use loopback dev 1! direwolf is connected to dev 0.
loop_idx_rcv = 2 # Use loopback dev 0. 
sd.default.samplerate = 48000
sd.default.channels = 1

In [7]:
# Enter your address here
myaddress = "Cory Hall Berkeley CA USA"

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="EE123")
location = geolocator.geocode(myaddress)

In [8]:
# hopefully it did not fail
print(location.address)
print((location.latitude, location.longitude))

Cory Hall, A. Richard Newton Bridge, Northside, Berkeley, Alameda County, California, 94720, United States
(37.87505015, -122.25784108556292)


In [9]:
import math
lat = "lat=%d^%2.2f%s" % (int(abs(location.latitude)),
                          60*(abs(location.latitude) - abs(math.floor(location.latitude))),
                          ("N") if location.latitude >0 else "S" )  
long = "long=%d^%2.2f%s" % (int(abs(location.longitude)),
                            60*(abs(location.longitude) - math.floor(abs(location.longitude))),
                          ("W") if location.longitude <0 else "E" )  
                            
print(lat, long)

lat=37^52.50N long=122^15.47W


In [10]:
# enter your callsign and comment for the beacon
callsign = "EE1233-8"
comment = "EE123 Final Project"

# generate passcode
callsignr = callsign.split('-')[0].upper()
code = 0x73e2
for i, char in enumerate(callsignr):
    code ^= ord(char) << (8 if not i % 2 else 0)

passcode = code & 0x7fff
print("Passcode:", passcode)

Passcode: 13478


In [11]:
import os

cmd = "cat direwolf-loopback-DRAFT.conf  | sed  's/EE123_CALL/"+callsign+"/g'  | "
cmd = cmd +  "sed  's/EE123_PASSCODE/%d/g' | " % (passcode)
cmd = cmd +  "sed  's/EE123_COMMENT/comment=\"%s\"/g' | " % (comment)
cmd = cmd +  "sed  's/EE123_LATCOORD/%s/g' | " % (lat)
cmd = cmd +  "sed  's/EE123_LONGCOORD/%s/g' > direwolf-loopback.conf" % (long)
print(cmd)
os.system(cmd)  # NOTE: this is poor security practice, but conveniently concise here

cat direwolf-loopback-DRAFT.conf  | sed  's/EE123_CALL/EE1233-8/g'  | sed  's/EE123_PASSCODE/13478/g' | sed  's/EE123_COMMENT/comment="EE123 Final Project"/g' | sed  's/EE123_LATCOORD/lat=37^52.50N/g' | sed  's/EE123_LONGCOORD/long=122^15.47W/g' > direwolf-loopback.conf


0

### Pre-task 4
Once you have edited the config file, start a terminal (either via SSH or in Jupyter from the directory page), change to this directory, and run
```sh
./run-direwolf.sh
```

Leave direwolf running while you are working on the lab. You can close it with control-c once you are finished.

* **Go to  http://igen2.eecs.berkeley.edu:14501 and make sure your callsign+ssid appears under clients.**

In [20]:
# png file selection
input_png = "Rolling_cal.png"
output_png = "reconstructed_cal.png"

In [ ]:
# perform processing, save output pngs
# current processing:
# 1. mirror pad, chroma downsampling
# 2. break into blocks
# 3. DCT-II Transform
# 4. quantization
# 5. zigzag, ZRLE
# 6. Huffman encoding
# 7. split into DC/AC blocks
# 8. reconstruct bits from DC/AC channels

In [22]:
# Load apng into individual png files
im = APNG.open(input_png)
input_files, output_files = [], []
for i, (png, control) in enumerate(im.frames):
    frame = "input_frames/{i}.png".format(i=i)
    input_files.append(frame)
    png.save(frame)

In [24]:
encoded_bits = []
num_frames = len(input_files)
for i in range(num_frames):
    img = Image.open(input_files[i])
    frame = "output_frames/{i}.png".format(i=i)
    
    
    output_files.append(frame)
    img = np.array(img)[:,:,0:3]
    M, N = img.shape[0:2]
    # encoding/decoding of image
    bits = encode_image(img, quality=75)
    encoded_bits.append(bits)

    img_dec = decode_image(bits, M, N, quality=75)
    pil_image = Image.fromarray(img_dec.astype(np.uint8),"RGB")
    pil_image = pil_image.save(frame)
    
APNG.from_files(output_files, delay=100).save(output_png)

In [25]:
# # generate bitstream to transmit
# print(np.shape(encoded_bits))
# bitstream = bitarray(format(M,'08b'))
# bitstream.extend(bitarray(format(N,'08b')))
# bitstream.extend(bitarray(format(num_frames,'08b')))

(13, 6)


In [26]:
# for i in range(num_frames):
#     for j in range(6):
#         bitstream.extend(bitarray(format(len(encoded_bits[i][j]), '16b')))
# for i in range(num_frames):
#     for j in range(6):
#         bitstream.extend(encoded_bits[i][j])

In [28]:
# print("bitstream",len(bitstream))
# base64 = ee123_bitarr_to_base64(bitstream)
# print(M*N*6*len(input_files))

In [32]:
# Enqueue and transmit images.

callsign = "EE1233-8"
partner_callsign = "NU6XB"
fs = 48000
modem = TNCaprs(fs=fs, Abuffer=1024, Nchunks=10)
print("Putting packets in Queue")

# Enqueue the files

# encode the file into a jpeg123 (files currently in input_files)
# For each file, use file_to_b64 to convert
# merge b64 convertions into a single data field


file_b64 = file_to_b64(input_files[0])

for i in range(1, len(input_files)):
    file_b64 = file_b64 + file_to_b64(input_files[i])

Qout, uid = enqueue_data(callsign=callsign, modem=modem, data=file_b64, uid=1010, fname=input_png)

print("Done. UID=%s" % uid.decode())
print(f"There are {Qout.qsize()} packets in Qout")

Putting packets in Queue
Putting packets in Queue for transmission ID=1010
Done. UID=1010
There are 486 packets in Qout


In [ ]:
#transmit packets to the server here

In [ ]:
def make_QR_callback(Qin):
    def queueREPLAY_callback(indata, outdata, frames, time, status):
#         assert frames == 1024
        if status:
            print(status)
        outdata[:] = indata
        Qin.put(indata.copy()[:, 0]) # Global queue
    return queueREPLAY_callback

In [33]:
# Now we will receive and reassemble a file transmitted over the loopback interface.

In [ ]:
# transmission
Qin = Queue.Queue()
print("Starting transmission")
starttime = time.time()
# Your code here: Modulate and send
while not Qout.empty():
    sd.play(Qout.get()*0.15, samplerate=48000, device=loop_idx, blocking=True)
    time.sleep(0.2)
# End of your code